In [2]:
import geocoder
import numpy as np
import pandas as pd
import datetime as dt

# Explore the Transactions

In [10]:
# Reading the file and skiping the first row which is just a note
transactions = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx', sheet_name = 'Transactions', skiprows=1)
transactions.head()

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25,0.0,Approved,Solex,Standard,medium,medium,71.49,53.62,41245.0
1,2,3,3120,2017-05-21,1.0,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701.0
2,3,37,402,2017-10-16,0.0,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361.0
3,4,88,3135,2017-08-31,0.0,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.10,36145.0
4,5,78,787,2017-10-01,1.0,Approved,Giant Bicycles,Standard,medium,large,1765.30,709.48,42226.0


In [11]:
# Check the info of the table
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 13 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   transaction_id           20000 non-null  int64         
 1   product_id               20000 non-null  int64         
 2   customer_id              20000 non-null  int64         
 3   transaction_date         20000 non-null  datetime64[ns]
 4   online_order             19640 non-null  float64       
 5   order_status             20000 non-null  object        
 6   brand                    19803 non-null  object        
 7   product_line             19803 non-null  object        
 8   product_class            19803 non-null  object        
 9   product_size             19803 non-null  object        
 10  list_price               20000 non-null  float64       
 11  standard_cost            19803 non-null  float64       
 12  product_first_sold_date  19803 n

- Columns with missing values:online order, brand, product_line, product_class, product_size, standard_cost, product_first_sold_date columns　　　

- Wrong data type: product_first_sold_date columns is in float, but it should be datatime　　


In [12]:
# convert the datatype of the product_first_sold_date to datatime
# https://stackoverflow.com/questions/38454403/convert-excel-style-date-with-pandas
transactions['product_first_sold_date'] = pd.TimedeltaIndex(transactions['product_first_sold_date'], unit='d')+ dt.datetime(1900,1,1)
transactions['product_first_sold_date'].head()

0   2012-12-04
1   2014-03-05
2   1999-07-22
3   1998-12-18
4   2015-08-12
Name: product_first_sold_date, dtype: datetime64[ns]

In [13]:
transactions['product_first_sold_date'].tail()

19995   2003-07-23
19996   1997-05-12
19997   2010-08-22
19998   2004-08-19
19999   1999-06-25
Name: product_first_sold_date, dtype: datetime64[ns]

In [14]:
transactions['product_first_sold_date'].describe()

count                   19803
unique                    100
top       1992-10-04 00:00:00
freq                      234
first     1991-01-23 00:00:00
last      2016-12-08 00:00:00
Name: product_first_sold_date, dtype: object

In [15]:
# check the distribution of the transaction date and product_first_sold_date
# https://stackoverflow.com/questions/38792122/how-to-group-and-count-rows-by-month-and-year-using-pandas/38792452
transactions['transaction_date'].groupby([transactions['transaction_date'].dt.year]).agg('count')

transaction_date
2017    20000
Name: transaction_date, dtype: int64

In [16]:
transactions['transaction_date'].groupby([transactions['transaction_date'].dt.month]).agg('count')

transaction_date
1     1682
2     1623
3     1656
4     1655
5     1685
6     1581
7     1717
8     1749
9     1572
10    1771
11    1665
12    1644
Name: transaction_date, dtype: int64

In [17]:
transactions['product_first_sold_date'].groupby([transactions['product_first_sold_date'].dt.year]).agg('count')

product_first_sold_date
1991.0    1170
1992.0     445
1993.0    1360
1994.0     568
1995.0     415
1996.0     380
1997.0    1001
1998.0     403
1999.0     806
2000.0     370
2001.0     196
2002.0     568
2003.0    1545
2004.0    1224
2005.0     826
2006.0     770
2007.0     420
2008.0     213
2009.0     421
2010.0     816
2011.0    1342
2012.0    1044
2013.0     617
2014.0     547
2015.0    1146
2016.0    1190
Name: product_first_sold_date, dtype: int64

In [18]:
transactions.describe()

,transaction_id,product_id,customer_id,online_order,list_price,standard_cost
count,20000.000000,20000.00000,20000.000000,19640.000000,20000.000000,19803.000000
mean,10000.500000,45.36465,1738.246050,0.500458,1107.829449,556.046951
std,5773.647028,30.75359,1011.951046,0.500013,582.825242,405.955660
min,1.000000,0.00000,1.000000,0.000000,12.010000,7.210000
25%,5000.750000,18.00000,857.750000,0.000000,575.270000,215.140000
50%,10000.500000,44.00000,1736.000000,1.000000,1163.890000,507.580000
75%,15000.250000,72.00000,2613.000000,1.000000,1635.300000,795.100000
max,20000.000000,100.00000,5034.000000,1.000000,2091.470000,1759.850000


In [19]:
# check missing values
transactions.isnull().sum()

transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64

In [20]:
# Find the unique values of online_order, which data type is float
transactions.online_order.unique()

array([ 0.,  1., nan])

In [71]:
transactions.order_status.unique()

array(['Approved', 'Cancelled'], dtype=object)

In [21]:
# check the unique values in each columns with missing values
transactions.brand.unique()

array(['Solex', 'Trek Bicycles', 'OHM Cycles', 'Norco Bicycles',
       'Giant Bicycles', 'WeareA2B', nan], dtype=object)

In [22]:
transactions.product_line.unique()

array(['Standard', 'Road', 'Mountain', 'Touring', nan], dtype=object)

In [23]:
transactions.product_class.unique()

array(['medium', 'low', 'high', nan], dtype=object)

In [24]:
transactions.product_size.unique()

array(['medium', 'large', 'small', nan], dtype=object)

- Data types of brand, product_line, product_class, product_size are strings, we may drop those missing values if they are unnecessary for our analysis. However, the standard_cost and product_first_sold_date are precise number or time, the further processing of those missing values will depend on objective of the analysis;   
- All the values of columns mentioned above are accurate, consistent and valid 

In [25]:
# Check the duplicates in the table
transactions[transactions.duplicated()].sum()

transaction_id             0.0
product_id                 0.0
customer_id                0.0
transaction_date           0.0
online_order               0.0
order_status               0.0
brand                      0.0
product_line               0.0
product_class              0.0
product_size               0.0
list_price                 0.0
standard_cost              0.0
product_first_sold_date    0.0
dtype: float64

In [72]:
# save the dataframe as csv file without index
transactions.to_csv('transactions.csv', index=False)

# Explore the NewCustomerList

In [29]:
# import the NewCustomerList and skip the first row which is note
new_customer_list = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='NewCustomerList', skiprows=1)
new_customer_list.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,...,state,country,property_valuation,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,...,QLD,Australia,6,0.56,0.7000,0.8750,0.743750,1,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,...,NSW,Australia,11,0.75,0.7500,0.9375,0.796875,1,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,...,VIC,Australia,5,0.65,0.6500,0.6500,0.650000,1,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,...,QLD,Australia,1,0.83,1.0375,1.0375,1.037500,4,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,...,NSW,Australia,9,0.96,0.9600,1.2000,1.200000,4,4,1.703125


In [30]:
# Drop Unnamed Columns which are not there in original table
cols = ['Unnamed: 16','Unnamed: 17','Unnamed: 18','Unnamed: 19','Unnamed: 20']
new_customer_list = new_customer_list.drop(cols, axis=1)
new_customer_list.head()

,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,owns_car,tenure,address,postcode,state,country,property_valuation,Rank,Value
0,Chickie,Brister,Male,86,1957-07-12,General Manager,Manufacturing,Mass Customer,N,Yes,14,45 Shopko Center,4500,QLD,Australia,6,1,1.718750
1,Morly,Genery,Male,69,1970-03-22,Structural Engineer,Property,Mass Customer,N,No,16,14 Mccormick Park,2113,NSW,Australia,11,1,1.718750
2,Ardelis,Forrester,Female,10,1974-08-28,Senior Cost Accountant,Financial Services,Affluent Customer,N,No,10,5 Colorado Crossing,3505,VIC,Australia,5,1,1.718750
3,Lucine,Stutt,Female,64,1979-01-28,Account Representative III,Manufacturing,Affluent Customer,N,Yes,5,207 Annamark Plaza,4814,QLD,Australia,1,4,1.703125
4,Melinda,Hadlee,Female,34,1965-09-21,Financial Analyst,Financial Services,Affluent Customer,N,No,19,115 Montana Place,2093,NSW,Australia,9,4,1.703125


In [28]:
# Check the info of the table
new_customer_list.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   first_name                           1000 non-null   object        
 1   last_name                            971 non-null    object        
 2   gender                               1000 non-null   object        
 3   past_3_years_bike_related_purchases  1000 non-null   int64         
 4   DOB                                  983 non-null    datetime64[ns]
 5   job_title                            894 non-null    object        
 6   job_industry_category                835 non-null    object        
 7   wealth_segment                       1000 non-null   object        
 8   deceased_indicator                   1000 non-null   object        
 9   owns_car                             1000 non-null   object        
 10  tenure       

**For this stage:**  

- columns with missing values: last_name, DOB, job_title, job_industry_category;   
- Wrong data type: postcode should convert from int to string.

In [31]:
# convert the int into string in postcode
new_customer_list['postcode'] = new_customer_list['postcode'].astype(str)
print(new_customer_list['postcode'].dtypes)

object


In [30]:
# Check for null values
new_customer_list.isnull().sum()

first_name                               0
last_name                               29
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     17
job_title                              106
job_industry_category                  165
wealth_segment                           0
deceased_indicator                       0
owns_car                                 0
tenure                                   0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
Rank                                     0
Value                                    0
dtype: int64

We will take the further step with the missing values based on the goal of final analysis.

In [31]:
# Check the duplicates in the table
new_customer_list[new_customer_list.duplicated()].sum()

first_name                             0.0
last_name                              0.0
gender                                 0.0
past_3_years_bike_related_purchases    0.0
DOB                                    0.0
job_title                              0.0
job_industry_category                  0.0
wealth_segment                         0.0
deceased_indicator                     0.0
owns_car                               0.0
tenure                                 0.0
address                                0.0
postcode                               0.0
state                                  0.0
country                                0.0
property_valuation                     0.0
Rank                                   0.0
Value                                  0.0
dtype: float64

There is no duplicates in the new_customer_list.   

In [32]:
# check the descriptive statistics of new_customer_list
new_customer_list.describe()

,past_3_years_bike_related_purchases,tenure,property_valuation,Rank,Value
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,49.836000,11.388000,7.397000,498.819000,0.881714
std,27.796686,5.037145,2.758804,288.810997,0.293525
min,0.000000,0.000000,1.000000,1.000000,0.340000
25%,26.750000,7.000000,6.000000,250.000000,0.649531
50%,51.000000,11.000000,8.000000,500.000000,0.860000
75%,72.000000,15.000000,9.000000,750.250000,1.075000
max,99.000000,22.000000,12.000000,1000.000000,1.718750


In [33]:
# check the validity and consistence of values in each columns
new_customer_list['DOB'].describe()

count                     983
unique                    958
top       1961-07-31 00:00:00
freq                        2
first     1938-06-08 00:00:00
last      2002-02-27 00:00:00
Name: DOB, dtype: object

In [34]:
# check the distribution of the DOB
new_customer_list['DOB'].groupby([new_customer_list['DOB'].dt.year]).agg('count')

DOB
1938.0    11
1939.0    13
1940.0    13
1941.0     9
1942.0     8
          ..
1998.0    14
1999.0    19
2000.0    11
2001.0     9
2002.0     3
Name: DOB, Length: 65, dtype: int64

In [35]:
new_customer_list['gender'].value_counts()

Female    513
Male      470
U          17
Name: gender, dtype: int64

In [36]:
new_customer_list['job_title'].unique()

array(['General Manager', 'Structural Engineer', 'Senior Cost Accountant',
       'Account Representative III', 'Financial Analyst',
       'Assistant Media Planner', 'Compensation Analyst',
       'Human Resources Assistant II', 'Speech Pathologist',
       'Accounting Assistant IV', 'Programmer Analyst I',
       'Environmental Specialist', 'Human Resources Manager',
       'VP Marketing', 'Clinical Specialist', nan, 'Associate Professor',
       'Actuary', 'Internal Auditor', 'Programmer I', 'Quality Engineer',
       'Quality Control Specialist', 'Account Executive', 'Engineer II',
       'Nurse', 'Librarian', 'Paralegal', 'Assistant Manager',
       'Senior Quality Engineer', 'Recruiting Manager', 'Design Engineer',
       'Business Systems Development Analyst', 'VP Quality Control',
       'Cost Accountant', 'Product Engineer', 'Social Worker',
       'Payment Adjustment Coordinator', 'Marketing Assistant',
       'Recruiter', 'VP Product Management', 'Media Manager III',
       

In [32]:
new_customer_list['job_industry_category'].value_counts()

Financial Services    203
Manufacturing         199
Health                152
Retail                 78
Property               64
IT                     51
Entertainment          37
Argiculture            26
Telecommunications     25
Name: job_industry_category, dtype: int64

In [33]:
new_customer_list['wealth_segment'].value_counts()

Mass Customer        508
High Net Worth       251
Affluent Customer    241
Name: wealth_segment, dtype: int64

In [39]:
new_customer_list['deceased_indicator'].unique()

array(['N'], dtype=object)

In [40]:
new_customer_list['owns_car'].unique()

array(['Yes', 'No'], dtype=object)

In [41]:
new_customer_list['address'].unique()

array(['45 Shopko Center', '14 Mccormick Park', '5 Colorado Crossing',
       '207 Annamark Plaza', '115 Montana Place', '89105 Pearson Terrace',
       '7 Nevada Crossing', '85 Carioca Point', '717 West Drive',
       '80 Scofield Junction', '3682 Crowley Point',
       '3 Golden Leaf Point', '0484 North Avenue', '0591 Anzinger Circle',
       '39 Kedzie Pass', '64 Granby Parkway', '610 Swallow Street',
       '61 4th Street', '1550 Russell Way', '193 North Point',
       '321 Raven Plaza', '656 Fuller Street', '1562 Merchant Street',
       '663 8th Parkway', '67 Shelley Street', '75 Cordelia Trail',
       '51 Hooker Court', '1859 Forest Circle', '44557 Rutledge Court',
       '11184 East Drive', '555 Hermina Avenue', '8 Novick Trail',
       '74 Welch Pass', '3 Pleasure Drive', '8 Dennis Point',
       '41042 Lotheville Crossing', '2330 Butternut Trail',
       '240 Acker Avenue', '04 Dexter Way', '011 Northland Trail',
       '8 Grayhawk Circle', '44 Darwin Lane', '2548 Arrowood P

In [34]:
new_customer_list['state'].value_counts()

NSW    506
VIC    266
QLD    228
Name: state, dtype: int64

In [43]:
new_customer_list['country'].unique()

array(['Australia'], dtype=object)

- All the values are valid, accurate and consistent, except for some missing values in last_name, DOB, job_title, job_industry_category. The number of missing values in DOB and last_name are in small amount, so we might drop them. However, the further step with missing values in other two columns will depend on the final goal of analysis;   
- Lack the explanation of collection of several columns, the meaning for each values. They are deceased_indicator, tenure, property_valuation, Rank and Value. Especially, for the U in gender column, we need to confirm its meaning before futher step;  
- All the values in country and deceased_indicator columns are same, seperately, the further step of these column will depend on the finl goal of analysis.

# Explore the CustomerDemographic

In [3]:
# import the NewCustomerList and skip the first row which is note
customer_demographic = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='CustomerDemographic', skiprows=1)
customer_demographic.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11.0
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16.0
2,3,Arlin,Dearle,Male,61,1954-01-20,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15.0
3,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7.0
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8.0


In [4]:
# Check the info of the table
customer_demographic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4000 entries, 0 to 3999
Data columns (total 13 columns):
customer_id                            4000 non-null int64
first_name                             4000 non-null object
last_name                              3875 non-null object
gender                                 4000 non-null object
past_3_years_bike_related_purchases    4000 non-null int64
DOB                                    3913 non-null datetime64[ns]
job_title                              3494 non-null object
job_industry_category                  3344 non-null object
wealth_segment                         4000 non-null object
deceased_indicator                     4000 non-null object
default                                3698 non-null object
owns_car                               4000 non-null object
tenure                                 3913 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(2), object(9)
memory usage: 406.3+ KB


For this stage:

- columns with missing values: last_name, DOB, job_title, job_industry_category, default, tenure;   

- columns that lack data explanation: deceased_indicator, default and tenure;   
- values in default and gender columns look tanglesome.

In [5]:
# check the values in default column
customer_demographic['default'].unique()

array(['"\'', "<script>alert('hi')</script>",
       datetime.datetime(2018, 2, 1, 0, 0),
       '() { _; } >_[$($())] { touch /tmp/blns.shellshock2.fail; }',
       'NIL', 'ðµ ð ð ð', 'â°â´âµâââ', '(â¯Â°â¡Â°ï¼â¯ï¸µ â»ââ»)', '0/0',
       'ð©ð½', 'ÅâÂ´Â®â\xa0Â¥Â¨ËÃ¸Ïââ', 'nil', -100, 'â°â´âµ', 'ð',
       1000000000000000049861653971908893017010268485438462151574892930611988399099305815384459015356416,
       'ï¾ï½¥â¿ã¾â²(ï½¡ââ¿âï½¡)â±â¿ï½¥ï¾', 'Î©âÃ§ââ«ËÂµâ¤â¥Ã·',
       'ÅâÂ´â°ËÃÂ¨ËÃâââ', 'ï¼ï¼ï¼',
       '../../../../../../../../../../../etc/hosts',
       '×Ö¸×Ö°×ªÖ¸×testØ§ÙØµÙØ\xadØ§Øª Ø§ÙØªÙØ\xadÙÙ', '<>?:"{}|_+',
       '\'\'\'\'"', ",./;'[]\\-=",
       '() { 0; }; touch /tmp/blns.shellshock1.fail;',
       'ì¬íê³¼íì ì´íì°êµ¬ì', 'testâ\xa0testâ«',
       '0ï¸â£ 1ï¸â£ 2ï¸â£ 3ï¸â£ 4ï¸â£ 5ï¸â£ 6ï¸â£ 7ï¸â£ 8ï¸â£ 9ï¸â£ ð',
       nan, '!@#$%^&*()', "'",
       'Ì¦HÍÌ¬Ì¤ÌÌ¤eÍ ÍÌÌ¥ÌÌ»ÍÌwÌhÌÌ¯ÍoÌÍÌÍÌ±Ì® ÒÌºÌÌÌÍWÌ·Ì¼Ì\xadaÌºÌªÍiÌ¨ÍÍÌ\xadÍÌ¯ÌtÌ¶Ì¼Ì®sÌÌÍÍ Ì\xa0Ì«Ì\xa0BÌ»ÍÍÍÍÌ³eÌµhÌµÌ¬ÍÌ«Í

We have to later discuss with the auther of the customer_demographic table about the default column. Then we will take further step depends on the goal of our analysis.

In [6]:
# Check for null values
customer_demographic.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64

We will take the further step with the missing values based on the goal of final analysis.

In [7]:
# Check the duplicates in the table
customer_demographic[customer_demographic.duplicated()].sum()

customer_id                            0.0
first_name                             0.0
last_name                              0.0
gender                                 0.0
past_3_years_bike_related_purchases    0.0
DOB                                    0.0
job_title                              0.0
job_industry_category                  0.0
wealth_segment                         0.0
deceased_indicator                     0.0
default                                0.0
owns_car                               0.0
tenure                                 0.0
dtype: float64

There is no duplicates in the customer_demographic.

In [8]:
# check the descriptive statistics of customer_demographic
customer_demographic.describe()

,customer_id,past_3_years_bike_related_purchases,tenure
count,4000.000000,4000.000000,3913.000000
mean,2000.500000,48.890000,10.657041
std,1154.844867,28.715005,5.660146
min,1.000000,0.000000,1.000000
25%,1000.750000,24.000000,6.000000
50%,2000.500000,48.000000,11.000000
75%,3000.250000,73.000000,15.000000
max,4000.000000,99.000000,22.000000


In [9]:
# check the validity and consistence of values in each columns
customer_demographic['DOB'].describe()

count                    3913
unique                   3448
top       1978-01-30 00:00:00
freq                        7
first     1843-12-21 00:00:00
last      2002-03-11 00:00:00
Name: DOB, dtype: object

In [10]:
# check the distribution of the DOB
customer_demographic['DOB'].groupby([customer_demographic['DOB'].dt.year]).agg('count')

DOB
1843.0      1
1931.0      1
1935.0      1
1940.0      1
1943.0      1
1944.0      1
1953.0     19
1954.0     61
1955.0     58
1956.0     63
1957.0     71
1958.0     55
1959.0     94
1960.0     58
1961.0     60
1962.0     76
1963.0     59
1964.0     85
1965.0     69
1966.0     65
1967.0     73
1968.0     69
1969.0     78
1970.0     57
1971.0     88
1972.0     54
1973.0    122
1974.0    152
1975.0    114
1976.0    152
1977.0    204
1978.0    217
1979.0    118
1980.0    123
1981.0     79
1982.0     41
1983.0     40
1984.0     43
1985.0     91
1986.0    114
1987.0     72
1988.0     72
1989.0     74
1990.0     53
1991.0     48
1992.0     74
1993.0     60
1994.0     79
1995.0     90
1996.0     67
1997.0     74
1998.0     80
1999.0     60
2000.0     42
2001.0     34
2002.0      6
Name: DOB, dtype: int64

In [11]:
customer_demographic['gender'].value_counts()

Female    2037
Male      1872
U           88
Femal        1
M            1
F            1
Name: gender, dtype: int64

In [12]:
# replace the other values in gender into appropriate values
#https://www.kite.com/python/answers/how-to-replace-column-values-in-a-pandas-dataframe-in-python
customer_demographic['gender'].replace({'M':'Male', 'F': 'Female', 'Femal': 'Female'}, inplace=True)
customer_demographic['gender'].value_counts()

Female    2039
Male      1873
U           88
Name: gender, dtype: int64

In [13]:
customer_demographic['job_title'].unique()

array(['Executive Secretary', 'Administrative Officer',
       'Recruiting Manager', nan, 'Senior Editor', 'Media Manager I',
       'Business Systems Development Analyst', 'Senior Quality Engineer',
       'Nuclear Power Engineer', 'Developer I', 'Account Executive',
       'Junior Executive', 'Media Manager IV', 'Sales Associate',
       'Professor', 'Geological Engineer', 'Project Manager',
       'Safety Technician I', 'Research Assistant I',
       'Accounting Assistant III', 'Editor', 'Research Nurse',
       'Safety Technician III', 'Staff Accountant III', 'Legal Assistant',
       'Product Engineer', 'Information Systems Manager',
       'VP Quality Control', 'Social Worker', 'Senior Cost Accountant',
       'Assistant Media Planner', 'Payment Adjustment Coordinator',
       'Food Chemist', 'Accountant III', 'Director of Sales',
       'Senior Financial Analyst', 'Registered Nurse',
       'Biostatistician II', 'Computer Systems Analyst II',
       'Software Test Engineer II', 

In [14]:
customer_demographic['job_industry_category'].value_counts()

Manufacturing         799
Financial Services    774
Health                602
Retail                358
Property              267
IT                    223
Entertainment         136
Argiculture           113
Telecommunications     72
Name: job_industry_category, dtype: int64

In [15]:
customer_demographic['wealth_segment'].value_counts()

Mass Customer        2000
High Net Worth       1021
Affluent Customer     979
Name: wealth_segment, dtype: int64

In [16]:
customer_demographic['deceased_indicator'].unique()

array(['N', 'Y'], dtype=object)

In [17]:
customer_demographic['owns_car'].unique()

array(['Yes', 'No'], dtype=object)

- All the values are valid, accurate and consistent, except for some missing values in last_name, DOB, job_title, job_industry_category, default and tenure, as well as the DOB in 1843. Especially, all the values in default column are unreadable. The further step with missing values in these columns will depend on the final goal of analysis;
- Lack the explanation of collection of several columns, the meaning for each values. They are deceased_indicator, tenure, and default. Especially, for the U in gender column, we need to confirm its meaning before futher step.

# Explore the CustomerAddress

In [18]:
# import the CustomerAddress and skip the first row which is note
customer_address = pd.read_excel('KPMG_VI_New_raw_data_update_final.xlsx',sheet_name='CustomerAddress', skiprows=1)
customer_address.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [19]:
# Check the info of the table
customer_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3999 entries, 0 to 3998
Data columns (total 6 columns):
customer_id           3999 non-null int64
address               3999 non-null object
postcode              3999 non-null int64
state                 3999 non-null object
country               3999 non-null object
property_valuation    3999 non-null int64
dtypes: int64(3), object(3)
memory usage: 187.5+ KB


For this stage:

- No missing values;

- postcode date type should be string, not the integer.

In [20]:
# convert the date type of postcode to string 
customer_address['postcode'] = customer_address['postcode'].astype(str)
print(customer_address['postcode'].dtypes)

object


In [21]:
# Check the duplicates in the table
customer_address[customer_address.duplicated()].sum()

customer_id           0.0
address               0.0
postcode              0.0
state                 0.0
country               0.0
property_valuation    0.0
dtype: float64

In [22]:
# check the descriptive statistics of CustomerAddress
customer_address.describe()

,customer_id,property_valuation
count,3999.000000,3999.000000
mean,2003.987997,7.514379
std,1154.576912,2.824663
min,1.000000,1.000000
25%,1004.500000,6.000000
50%,2004.000000,8.000000
75%,3003.500000,10.000000
max,4003.000000,12.000000


In [23]:
# check the validity and consistence of values in each columns
customer_address['address'].unique()

array(['060 Morning Avenue', '6 Meadow Vale Court', '0 Holy Cross Court',
       ..., '87 Crescent Oaks Alley', '8194 Lien Street',
       '320 Acker Drive'], dtype=object)

In [24]:
customer_address['state'].unique()

array(['New South Wales', 'QLD', 'VIC', 'NSW', 'Victoria'], dtype=object)

In [25]:
# replace the other values in state into appropriate values
customer_address['state'].replace({'New South Wales':'NSW', 'Victoria': 'VIC'}, inplace=True)
customer_address['state'].value_counts()

NSW    2140
VIC    1021
QLD     838
Name: state, dtype: int64

In [26]:
customer_address['country'].unique()

array(['Australia'], dtype=object)

In [40]:
len(customer_address['postcode'].unique())

873

- All the values are valid, accurate and consistent. No missing values and duplicates;
- All values in country column are same, the further step of it depends on the goal of our analysis;   
- customer_address could merge with customer_demographic since they share the same customer_id.

# Merge the CustomerAddress and CustomerDemographic

In [27]:
# merge two dataframes as one on the same customer_id
customer_df = pd.merge(customer_demographic, customer_address, how='inner', on='customer_id')
customer_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3996 entries, 0 to 3995
Data columns (total 18 columns):
customer_id                            3996 non-null int64
first_name                             3996 non-null object
last_name                              3871 non-null object
gender                                 3996 non-null object
past_3_years_bike_related_purchases    3996 non-null int64
DOB                                    3909 non-null datetime64[ns]
job_title                              3492 non-null object
job_industry_category                  3341 non-null object
wealth_segment                         3996 non-null object
deceased_indicator                     3996 non-null object
default                                3694 non-null object
owns_car                               3996 non-null object
tenure                                 3909 non-null float64
address                                3996 non-null object
postcode                               3996 no

There are in total 3996 rows after mergeing in the new customer dataframe.

In [28]:
# Check for null values
customer_df.isnull().sum()

customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              504
job_industry_category                  655
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
dtype: int64

In [29]:
# Check the duplicates in the table
customer_df[customer_df.duplicated()].sum()

customer_id                            0.0
first_name                             0.0
last_name                              0.0
gender                                 0.0
past_3_years_bike_related_purchases    0.0
DOB                                    0.0
job_title                              0.0
job_industry_category                  0.0
wealth_segment                         0.0
deceased_indicator                     0.0
default                                0.0
owns_car                               0.0
tenure                                 0.0
address                                0.0
postcode                               0.0
state                                  0.0
country                                0.0
property_valuation                     0.0
dtype: float64

In [30]:
customer_df[customer_df['DOB'].dt.year == 1843]

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure,address,postcode,state,country,property_valuation
29,34,Jephthah,Bachmann,U,59,1843-12-21,Legal Assistant,IT,Affluent Customer,N,NaN,No,20.0,833 Luster Way,4005,QLD,Australia,8


In [31]:
# drop the row which DOB is in 1843
customer_df = customer_df.drop(29)
customer_df['DOB'].describe()

count                    3908
unique                   3443
top       1978-01-30 00:00:00
freq                        7
first     1931-10-23 00:00:00
last      2002-03-11 00:00:00
Name: DOB, dtype: object

In [32]:
customer_df['owns_car'].value_counts()

Yes    2021
No     1974
Name: owns_car, dtype: int64

In [37]:
# convert postcode to location information 
def get_geocoder(postal_code_from_df):
    g = geocoder.osm('{}, Australia'.format(postal_code_from_df))
    return g.json["lat"],g.json["lng"]

In [42]:
code_map = {}
for postcode in customer_address['postcode'].unique():
    lat, lng = get_geocoder(postcode)
    code_map[postcode] = (lat, lng)

In [48]:
len(code_map)
def helper(x):
    return code_map[x][0], code_map[x][1]

In [50]:
customer_df['latitude'], customer_df['longitude'] = zip(*customer_df['postcode'].apply(helper))
customer_df

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,...,tenure,address,postcode,state,country,property_valuation,Latitude,Longitude,latitude,longitude
0,1,Laraine,Medendorp,Female,93,1953-10-12,Executive Secretary,Health,Mass Customer,N,...,11.0,060 Morning Avenue,2016,NSW,Australia,10,-33.893169,151.203612,-33.893169,151.203612
1,2,Eli,Bockman,Male,81,1980-12-16,Administrative Officer,Financial Services,Mass Customer,N,...,16.0,6 Meadow Vale Court,2153,NSW,Australia,10,-33.753486,150.998426,-33.753486,150.998426
2,4,Talbot,NaN,Male,33,1961-10-03,NaN,IT,Mass Customer,N,...,7.0,0 Holy Cross Court,4211,QLD,Australia,9,-27.995496,153.325939,-27.995496,153.325939
3,5,Sheila-kathryn,Calton,Female,56,1977-05-13,Senior Editor,NaN,Affluent Customer,N,...,8.0,17979 Del Mar Point,2448,NSW,Australia,4,-30.605714,153.001570,-30.605714,153.001570
4,6,Curr,Duckhouse,Male,35,1966-09-16,NaN,Retail,High Net Worth,N,...,13.0,9 Oakridge Court,3216,VIC,Australia,9,-38.185064,144.343247,-38.185064,144.343247
5,7,Fina,Merali,Female,6,1976-02-23,NaN,Financial Services,Affluent Customer,N,...,11.0,4 Delaware Trail,2210,NSW,Australia,9,-33.950310,151.057598,-33.950310,151.057598
6,8,Rod,Inder,Male,31,1962-03-30,Media Manager I,NaN,Mass Customer,N,...,7.0,49 Londonderry Lane,2650,NSW,Australia,4,-35.120263,147.381280,-35.120263,147.381280
7,9,Mala,Lind,Female,97,1973-03-10,Business Systems Development Analyst,Argiculture,Affluent Customer,N,...,8.0,97736 7th Trail,2023,NSW,Australia,12,-33.884284,151.261032,-33.884284,151.261032
8,11,Uriah,Bisatt,Male,99,1954-04-30,NaN,Property,Mass Customer,N,...,9.0,93405 Ludington Park,3044,VIC,Australia,8,-37.745018,144.942783,-37.745018,144.942783
9,12,Sawyere,Flattman,Male,58,1994-07-21,Nuclear Power Engineer,Manufacturing,Mass Customer,N,...,8.0,44339 Golden Leaf Alley,4557,QLD,Australia,4,-26.689711,153.106635,-26.689711,153.106635


In [51]:
location_map = customer_df[['customer_id','postcode','state','latitude','longitude']]
location_map.head()

,customer_id,postcode,state,latitude,longitude
0,1,2016,NSW,-33.893169,151.203612
1,2,2153,NSW,-33.753486,150.998426
2,4,4211,QLD,-27.995496,153.325939
3,5,2448,NSW,-30.605714,153.001570
4,6,3216,VIC,-38.185064,144.343247


In [52]:
location_map.to_csv('location.csv', index=False)

In [25]:
# save the dataframe as csv file without index
customer_df.to_csv('customer.csv', index=False)